# チューニング【画像の追加】
画像のパターンを増やすため、再度撮影を行いモデルに追加学習をさせる
## スマホにて画像撮影を実施し、該当イメージをアップロードする
* アップロードディレクトリを作成する

In [ ]:
import sagemaker, json, numpy as np, os, boto3, uuid
from PIL import Image, ImageDraw, ImageOps, ImageColor, ImageFont
import matplotlib.patches as patches
from matplotlib import pyplot as plt
from glob import glob

In [ ]:
import glob
import re
def make_dir_by_seq(dirname, seq_digit=3):
    pattern = f"{os.path.basename(dirname)}_([0-9]*)"
    prog = re.compile(pattern)
    
    files = glob.glob(f"{dirname}_[0-9]*", recursive=True)
    
    max_seq = -1
    for f in files:
        m = prog.match(os.path.basename(f))
        if m:
            max_seq = max(max_seq, int(m.group(1)))
    
    new_filename = f"{dirname}_{max_seq+1:0{seq_digit}}"
    
    return new_filename

newdir = make_dir_by_seq("./train_raw_images")
print(newdir)
os.makedirs(newdir, exist_ok=True)


* Studioのアップロードアイコンをクリックし新しく作成したフォルダへアップロードする
* ファイルフォーマットの変換及び適正サイズへリサイズを実施する
    * IMAGE_PATHを新しく作成したフォルダへ変更
        * 例） IMAGE_PATH="train_raw_images_000"

In [ ]:
# format変換及びリサイズ
## アスペクト比を固定して、幅が指定した値になるようリサイズする。
IMAGE_PATH="train_raw_images_000"

def scale_to_width(img, width):
    height = round(img.height * width / img.width)
    return img.resize((width, height))

def convHeic2Png(image_path):
    new_name = image_path.replace('HEIC', 'png')
    heif_file = pyheif.read(image_path)
    data = Image.frombytes(
        heif_file.mode,
        heif_file.size,
        heif_file.data,
        "raw",
        heif_file.mode,
        heif_file.stride,
        )
    # 画像をリサイズする
    data_resized = scale_to_width(data, 500)
    data_resized.save(new_name, "PNG")
    print(f'output heic -> png file:{new_name}')
    
def convJpeg2Png(image_path):
    new_name = image_path.replace('jpeg', 'png')
    img = Image.open(image_path)
    # 画像をリサイズする
    img_resized = scale_to_width(img, 500)
    img_resized.save(new_name, "PNG")
    print(f'output jpeg -> png file:{new_name}')

def convPng2Png(image_path):
    new_name = image_path
    img = Image.open(image_path)
    # 画像をリサイズする
    img_resized = scale_to_width(img, 500)
    img_resized.save(new_name, "PNG")
    print(f'output png -> png file:{new_name}')

print('データ変換開始')

lst = glob(f"./{IMAGE_PATH}/*.*")
for l in lst:
    img = Image.open(l)
    if l.endswith('heic'):
        convHeic2Png(l)
        os.remove(l)
    elif l.endswith('jpeg'):
        convJpeg2Png(l)
        os.remove(l)
    elif l.endswith('png'):
        convPng2Png(l)
        os.remove(l)
    else:
        print('ファイルタイプエラー')
print('データ変換終了')


* パラーメーターを指定する
    * <ユーザーID>を自分のユーザーIDへ置き換える
        * 例）　S3_BUCKET='training-user1-s3'
    * <イメージpath>を作成したフォルダへ変更
        * 例） IMAGES_PATH='training2/train_raw_images_000'

In [ ]:
# 各自のS3バケットを指定する
S3_BUCKET='training-menter1-s3'
# 【変更不要】imageファイルの格納場所指定
IMAGES_PATH='training2/train_raw_images_000'

* S3へアップロードする

In [ ]:
!aws s3 cp ./{IMAGE_PATH} s3://{S3_BUCKET}/{IMAGES_PATH}/images --exclude ".*"  --recursive

## 画像にラベリングを行う
* 初期と同様にラベリングジョブの作成
* ラベリング実施
* ラベリング結果をモデル作成用に加工 次のスクリプトを実行し各自用のS3バケットへ変換した結果をアップロードする
    * <ジョブ名>を先程実行したジョブ名に置き換えて実行する
       * 例） JOB_NAME="team-z-job2"


In [ ]:
# ラベリングを行ったジョブ名を指定してください
JOB_NAME="team-z-job6"

In [ ]:
OUTPUT_MANIFEST_PATH=''.join([IMAGES_PATH,'/images/',JOB_NAME,'/manifests/output/output.manifest'])
s3 = boto3.resource('s3')
my_bucket = s3.Bucket(S3_BUCKET)
output_manifest_object = my_bucket.Object(OUTPUT_MANIFEST_PATH)
output_manifest_json = output_manifest_object.get()["Body"].read().decode('utf-8').split()
OUTPUT_PATH = ''.join([IMAGES_PATH,'/','annotations.json'])


# 結果のきのこの山やたけのこの里の位置情報を格納する辞書 
annotation_dict = {
    'images':[],
    'annotations':[]
}

# 画像のファイル名に使う一意なシーケンス番号
IMAGE_ID = 0

# ラベリング結果の行数分ループする
# ラベリング結果は 1 行につき 1 画像格納される
for manifest_line in output_manifest_json:
    # 画像のラベリング結果の読み込み
    manifest_dict = json.loads(manifest_line)
    # print(manifest_dict)
    # 画像のファイル名取得(ラベリング結果に格納されている)
    filename = manifest_dict['source-ref'].split('/')[-1]
    annotation_list = manifest_dict[JOB_NAME]['annotations']
    # print(annotation_list)
    # ラベリング結果を出力用辞書に格納
    annotation_dict['images'].append(
        {
            'file_name' : filename,
            'height' : manifest_dict[JOB_NAME]['image_size'][0]['height'],
            'width' : manifest_dict[JOB_NAME]['image_size'][0]['width'],
            'id' : IMAGE_ID
        }
    )
    for annotation in annotation_list:
        # 座標変換
        left = annotation['left']
        top = annotation['top']
        right = annotation['left'] + annotation['width']
        bottom = annotation['top'] + annotation['height']
        # アノテーションを編集
        annotation_dict['annotations'].append(
            {
                'image_id': IMAGE_ID,
                'bbox': [left, top, right, bottom],
                # 文字列へ変換し格納
                # 'category_id': manifest_dict['kinotake-user1-metadata']['class-map'][str(annotation['class_id'])]
                'category_id': annotation['class_id']
            }
        )
    
    IMAGE_ID += 1
    
obj = s3.Object(S3_BUCKET, OUTPUT_PATH)
obj.put(Body = json.dumps(annotation_dict, ensure_ascii=False)) #←変数をJSON変換し S3にPUTする

In [ ]:
# モデルに対し再学習させる